In [183]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation
from matplotlib.patches import Circle
import seaborn as sns
from tqdm import tqdm
from mplsoccer import Pitch
import warnings
import ast
warnings.filterwarnings("ignore")

In [184]:
def calculate_position(x, y, distance, angle_rad):
    """
    Calculate the (x, y) position of loc1 given the distance and angle from loc2.
    
    Parameters:
    loc2 (tuple): The reference point (x2, y2).
    distance (float): The distance between loc1 and loc2.
    angle (float): The angle between the line connecting loc1 and loc2 and the positive x-axis.
    
    Returns:
    tuple: The (x, y) coordinates of loc1.
    """
    # Calculate the x and y coordinates
    new_x = x + distance * np.cos(angle_rad)
    new_y = y + distance * np.sin(angle_rad)

    return [new_x, new_y]

In [185]:
action_df = pd.read_csv('../data/processed/clean_action_data_glob.csv')
genetic_df = pd.read_csv('../src/modelling/genetic/results/result_each_gen.csv')

action_df['pred_base_xg'] = action_df['pred_base_xg'].apply(lambda x:np.round(x, 8))
genetic_df['initial_xg'] = genetic_df['initial_xg'].astype(float)

In [186]:
complete_df = genetic_df.merge(action_df, left_on = 'initial_xg', right_on = 'pred_base_xg', how = 'inner')

In [210]:
complete_df.head()

,index,idx_generation,initial_xg,best_xG,shot_statsbomb_xg,shot_x,shot_y,fk_x,fk_y,teammates_player_1,...,pos_opponent_3_x,pos_opponent_3_y,pos_opponent_4_x,pos_opponent_4_y,pos_opponent_5_x,pos_opponent_5_y,pos_opponent_6_x,pos_opponent_6_y,pos_opponent_7_x,pos_opponent_7_y
0,628,1,0.079421,0.104429,0.163926,111.0,44.0,95.0,62.0,0,...,109.697425,48.037734,103.806564,48.610258,117.726752,49.267903,118.543885,39.194816,121.770385,41.550754
1,628,2,0.079421,0.142249,0.163926,111.0,44.0,95.0,62.0,0,...,109.697425,48.037734,103.806564,48.610258,117.726752,49.267903,118.543885,39.194816,121.770385,41.550754
2,628,3,0.079421,0.162827,0.163926,111.0,44.0,95.0,62.0,0,...,109.697425,48.037734,103.806564,48.610258,117.726752,49.267903,118.543885,39.194816,121.770385,41.550754
3,628,4,0.079421,0.162827,0.163926,111.0,44.0,95.0,62.0,0,...,109.697425,48.037734,103.806564,48.610258,117.726752,49.267903,118.543885,39.194816,121.770385,41.550754
4,628,5,0.079421,0.164526,0.163926,111.0,44.0,95.0,62.0,0,...,109.697425,48.037734,103.806564,48.610258,117.726752,49.267903,118.543885,39.194816,121.770385,41.550754


In [188]:
complete_df = complete_df[['index', 'idx_generation', 'best_solution', 'initial_xg', 'best_xG', 'shot_statsbomb_xg', 
    'shot_x', 'shot_y', 'fk_x', 'fk_y', 'distance_player_1', 'distance_player_2', 'distance_player_3', 'distance_player_4', 'distance_player_5', 'distance_player_6',
       'distance_player_7', 'distance_player_8', 'distance_player_9', 'distance_player_10', 
       'angle_player_1', 'angle_player_2', 'angle_player_3', 'angle_player_4', 'angle_player_5', 'angle_player_6', 'angle_player_7', 'angle_player_8', 'angle_player_9', 'angle_player_10',
       'teammates_player_1', 'teammates_player_2', 'teammates_player_3', 'teammates_player_4', 'teammates_player_5', 'teammates_player_6',
       'teammates_player_7', 'teammates_player_8', 'teammates_player_9', 'teammates_player_10', 'pred_base_xg', 'pred_improved_xg', 'pctge_improvement']]

In [189]:
# retrieve all players position
for i in tqdm(range(1, 11)):
    complete_df[f'position_player_{i}'] = [calculate_position(complete_df['shot_x'][j], 
                                                              complete_df['shot_y'][j], 
                                                              complete_df[f'distance_player_{i}'][j], 
                                                              complete_df[f'angle_player_{i}'][j]) 
                                            for j in range(len(complete_df))]
    complete_df.drop([f'distance_player_{i}', f'angle_player_{i}'], axis = 1, inplace = True)

100%|██████████| 10/10 [00:00<00:00, 295.74it/s]


In [190]:
# keep only opponents fixed pos
for i in tqdm(range(1, 11)):
    if complete_df[f'teammates_player_{i}'][0] == 1:
        complete_df.drop([f'position_player_{i}', f'teammates_player_{i}'], axis = 1, inplace = True)

100%|██████████| 10/10 [00:00<00:00, 657.58it/s]


In [191]:
# format best solution
complete_df['best_solution'] = [complete_df['best_solution'][j].replace('\n', '').replace(' ', ', ') for j in range(len(complete_df))]
complete_df['best_solution'] = [ast.literal_eval(complete_df['best_solution'][j]) for j in range(len(complete_df))]

complete_df['shot_pos_gen_x'] = [complete_df['best_solution'][j][0] for j in range(len(complete_df))]
complete_df['shot_pos_gen_y'] = [complete_df['best_solution'][j][1] for j in range(len(complete_df))]

complete_df['pos_teammate_1_x'] = [complete_df['best_solution'][j][2] for j in range(len(complete_df))]
complete_df['pos_teammate_2_x'] = [complete_df['best_solution'][j][4] for j in range(len(complete_df))]
complete_df['pos_teammate_3_x'] = [complete_df['best_solution'][j][6] for j in range(len(complete_df))]
complete_df['pos_teammate_4_x'] = [complete_df['best_solution'][j][8] for j in range(len(complete_df))]
complete_df['pos_teammate_5_x'] = [complete_df['best_solution'][j][10] for j in range(len(complete_df))]

complete_df['pos_teammate_1_y'] = [complete_df['best_solution'][j][3] for j in range(len(complete_df))]
complete_df['pos_teammate_2_y'] = [complete_df['best_solution'][j][5] for j in range(len(complete_df))]
complete_df['pos_teammate_3_y'] = [complete_df['best_solution'][j][7] for j in range(len(complete_df))]
complete_df['pos_teammate_4_y'] = [complete_df['best_solution'][j][9] for j in range(len(complete_df))]
complete_df['pos_teammate_5_y'] = [complete_df['best_solution'][j][11] for j in range(len(complete_df))]

i=1
for col in complete_df.columns:
    if col.startswith('position'):
        complete_df[f'pos_opponent_{i}_x'] = [complete_df[col][j][0] for j in range(len(complete_df))]
        complete_df[f'pos_opponent_{i}_y'] = [complete_df[col][j][1] for j in range(len(complete_df))]
        complete_df.drop(col, axis = 1)
        i += 1

complete_df.drop(['best_solution'], axis = 1, inplace = True)

# format best_xg
complete_df['best_xG'] = complete_df['best_xG'].apply(lambda x : float(ast.literal_eval(x)[0]))

In [192]:
complete_df.head()

,index,idx_generation,initial_xg,best_xG,shot_statsbomb_xg,shot_x,shot_y,fk_x,fk_y,teammates_player_1,...,pos_opponent_3_x,pos_opponent_3_y,pos_opponent_4_x,pos_opponent_4_y,pos_opponent_5_x,pos_opponent_5_y,pos_opponent_6_x,pos_opponent_6_y,pos_opponent_7_x,pos_opponent_7_y
0,628,1,0.079421,0.104429,0.163926,111.0,44.0,95.0,62.0,0,...,109.697425,48.037734,103.806564,48.610258,117.726752,49.267903,118.543885,39.194816,121.770385,41.550754
1,628,2,0.079421,0.142249,0.163926,111.0,44.0,95.0,62.0,0,...,109.697425,48.037734,103.806564,48.610258,117.726752,49.267903,118.543885,39.194816,121.770385,41.550754
2,628,3,0.079421,0.162827,0.163926,111.0,44.0,95.0,62.0,0,...,109.697425,48.037734,103.806564,48.610258,117.726752,49.267903,118.543885,39.194816,121.770385,41.550754
3,628,4,0.079421,0.162827,0.163926,111.0,44.0,95.0,62.0,0,...,109.697425,48.037734,103.806564,48.610258,117.726752,49.267903,118.543885,39.194816,121.770385,41.550754
4,628,5,0.079421,0.164526,0.163926,111.0,44.0,95.0,62.0,0,...,109.697425,48.037734,103.806564,48.610258,117.726752,49.267903,118.543885,39.194816,121.770385,41.550754


In [193]:
action_id = 628
sub_df = complete_df[complete_df['index'] == action_id]
n_teammates = int(len([i for i in sub_df.columns if i.startswith('pos_teammate')])/2)
n_opponents = int(len([i for i in sub_df.columns if i.startswith('pos_opponent')])/2)

In [194]:
sub_df

,index,idx_generation,initial_xg,best_xG,shot_statsbomb_xg,shot_x,shot_y,fk_x,fk_y,teammates_player_1,...,pos_opponent_3_x,pos_opponent_3_y,pos_opponent_4_x,pos_opponent_4_y,pos_opponent_5_x,pos_opponent_5_y,pos_opponent_6_x,pos_opponent_6_y,pos_opponent_7_x,pos_opponent_7_y
0,628,1,0.079421,0.104429,0.163926,111.0,44.0,95.0,62.0,0,...,109.697425,48.037734,103.806564,48.610258,117.726752,49.267903,118.543885,39.194816,121.770385,41.550754
1,628,2,0.079421,0.142249,0.163926,111.0,44.0,95.0,62.0,0,...,109.697425,48.037734,103.806564,48.610258,117.726752,49.267903,118.543885,39.194816,121.770385,41.550754
2,628,3,0.079421,0.162827,0.163926,111.0,44.0,95.0,62.0,0,...,109.697425,48.037734,103.806564,48.610258,117.726752,49.267903,118.543885,39.194816,121.770385,41.550754
3,628,4,0.079421,0.162827,0.163926,111.0,44.0,95.0,62.0,0,...,109.697425,48.037734,103.806564,48.610258,117.726752,49.267903,118.543885,39.194816,121.770385,41.550754
4,628,5,0.079421,0.164526,0.163926,111.0,44.0,95.0,62.0,0,...,109.697425,48.037734,103.806564,48.610258,117.726752,49.267903,118.543885,39.194816,121.770385,41.550754
5,628,6,0.079421,0.165546,0.163926,111.0,44.0,95.0,62.0,0,...,109.697425,48.037734,103.806564,48.610258,117.726752,49.267903,118.543885,39.194816,121.770385,41.550754
6,628,7,0.079421,0.170322,0.163926,111.0,44.0,95.0,62.0,0,...,109.697425,48.037734,103.806564,48.610258,117.726752,49.267903,118.543885,39.194816,121.770385,41.550754
7,628,8,0.079421,0.170322,0.163926,111.0,44.0,95.0,62.0,0,...,109.697425,48.037734,103.806564,48.610258,117.726752,49.267903,118.543885,39.194816,121.770385,41.550754
8,628,9,0.079421,0.185049,0.163926,111.0,44.0,95.0,62.0,0,...,109.697425,48.037734,103.806564,48.610258,117.726752,49.267903,118.543885,39.194816,121.770385,41.550754
9,628,10,0.079421,0.185084,0.163926,111.0,44.0,95.0,62.0,0,...,109.697425,48.037734,103.806564,48.610258,117.726752,49.267903,118.543885,39.194816,121.770385,41.550754


In [195]:
# pitch = Pitch(pitch_type='statsbomb', pitch_color='grass', line_color='white')
# fig, ax = plt.subplots(figsize=(15,5))
# pitch.draw(ax=ax)

# anim = FuncAnimation(fig, update, frames=range(50), fargs=(complete_df, ax), interval=1000)

# plt.show()
# anim.save('scatter_animation.gif', writer='imagemagick')

In [201]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Function to create the football pitch using plotly_football_pitch
def create_pitch():
    
    dimensions = PitchDimensions(pitch_width_metres=80, pitch_length_metres=120)

    fig = make_pitch_figure(
        dimensions,
        pitch_background=SingleColourBackground("#3ab54a"),
    )

    fig.update_layout(
        xaxis=dict(range=[0, 120], showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(range=[0, 80], showgrid=False, zeroline=False, showticklabels=False),
        plot_bgcolor='#3ab54a',
        width=600, height=400,
        margin=dict(l=10, r=10, b=10, t=10)
    )

    return fig

# Function to add scatter points to the plot
def add_scatter(fig, data_row, show_legend=False):
    # Shot and free-kick positions
    fig.add_trace(go.Scatter(x=[data_row['shot_pos_gen_x']], y=[data_row['shot_pos_gen_y']], mode='markers',
                             marker=dict(size=10, symbol='star', color='black'), name='Shot Position', showlegend=show_legend))
    fig.add_trace(go.Scatter(x=[data_row['fk_x']], y=[data_row['fk_y']], mode='markers',
                             marker=dict(size=10, symbol='diamond', color='purple'), name='Freekick Position', showlegend=show_legend))

    # Opponents' positions
    for i in range(1, n_opponents):
        fig.add_trace(go.Scatter(x=[data_row[f'pos_opponent_{i}_x']], y=[data_row[f'pos_opponent_{i}_y']], mode='markers',
                                 marker=dict(size=8, symbol='x', color='orange'), name=f'Opponent', showlegend=show_legend and i == 1))

    # Teammates' positions
    for i in range(1, n_teammates):
        fig.add_trace(go.Scatter(x=[data_row[f'pos_teammate_{i}_x']], y=[data_row[f'pos_teammate_{i}_y']], mode='markers',
                                 marker=dict(size=8, symbol='circle', color='blue'), name=f'Teammate', showlegend=show_legend and i == 1))

    return fig

# Function to update the pitch with progress line and scatter points
def update_pitch(frame, df):
    data_row = df.iloc[frame]
    fig = create_pitch()

    # Add scatter points
    fig = add_scatter(fig, data_row, show_legend=(frame == 0))

    return fig

# Function to update the line plot with new points
def update_line_plot(frame, df, line_fig):
    if frame == 0:
        line_fig.add_trace(go.Scatter(x=[0], y=[df['best_xG'].iloc[0]], mode='lines+markers', name='Line Plot'))
    else:
        line_fig.data[0].x += (frame,)
        line_fig.data[0].y += (df['best_xG'].iloc[frame],)
    return line_fig


In [228]:
# Creating frames for animation
frames = []
pitch_fig = create_pitch()
line_fig = go.Figure()

for frame in range(len(sub_df)):
    pitch_fig = update_pitch(frame, sub_df)
    line_fig = update_line_plot(frame, sub_df, line_fig)
    
    frames.append(go.Frame(data=pitch_fig.data + line_fig.data, name=str(frame)))

# Creating the initial pitch and line plot
initial_pitch_fig = create_pitch()
initial_pitch_fig = add_scatter(initial_pitch_fig, sub_df.iloc[0], show_legend=True)
initial_line_fig = update_line_plot(0, sub_df, go.Figure())

# Adding frames to the figure for animation
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Football Pitch', 'Evolution of xG across Generations'),
    specs=[[{"type": "scatter"}, {"type": "scatter"}]]
)

# Add initial pitch and line plot
fig.add_traces(initial_pitch_fig.data, rows=1, cols=1)
fig.add_traces(line_fig.data, rows=1, cols=2)

fig.frames = frames

# Animation settings
fig.update_layout(
    updatemenus=[{
        "buttons": [
            {
                "args": [None, {"frame": {"duration": 500, "redraw": False}, "fromcurrent": True}],
                "label": "Play",
                "method": "animate"
            },
            {
                "args": [[None], {"frame": {"duration": 500, "redraw": False}, "mode": "immediate", "transition": {"duration": 0}}],
                "label": "Pause",
                "method": "animate"
            }
        ],
        "direction": "left",
        "pad": {"r": 10, "t": 87},
        "showactive": False,
        "type": "buttons",
        "x": 0.1,
        "xanchor": "right",
        "y": 0,
        "yanchor": "top"
    }],
    sliders=[{
        "yanchor": "top",
        "xanchor": "left",
        "currentvalue": {
            "font": {"size": 20},
            "prefix": "Frame:",
            "visible": True,
            "xanchor": "right"
        },
        "transition": {"duration": 500, "easing": "cubic-in-out"},
        "pad": {"b": 10, "t": 50},
        "len": 0.9,
        "x": 0.1,
        "y": 0,
        "steps": [{
            "args": [[str(k)], {"frame": {"duration": 500, "redraw": False}, "mode": "immediate",
                                    "transition": {"duration": 500}}],
            "label": str(k),
            "method": "animate",
        } for k in range(len(frames))]
    }],
    xaxis1=dict(range=[0, 120], showgrid=False, zeroline=False, showticklabels=False),
    yaxis1=dict(range=[0, 80], showgrid=False, zeroline=False, showticklabels=False),
    shapes=[
        dict(
            type="rect",
            xref="paper", yref="paper",
            x0=0, y0=0, x1=0.45, y1=1,
            fillcolor="#3ab54a",
            layer="below",
            line_width=0,
        ),
        dict(
            type="circle",
            xref="x1", yref="y1",  
            x0=60 - 10, y0=40 - 10, 
            x1=60 + 10, y1=40 + 10,  
            line_color="black",
            line_width=3
        )
    ],
    xaxis2=dict(tickvals=list(range(0, len(sub_df), 5)), ticktext=list(range(0, len(sub_df), 5))),
    xaxis2_title='Frame',
    yaxis2_title='xG',
    width=1600, 
    height=650,
)

fig.show()